## DIstributed Shared Nearest Neighbor experiments

In [ ]:
if length(workers()) > 1
    println("Removing previous workers...")
    rmprocs(workers())
end

addprocs(["158.251.93.5:3308","158.251.93.5:3307","158.251.93.5:3306","158.251.93.5:3305",
        "158.251.93.5:3304","158.251.93.5:3303","158.251.93.5:3302","158.251.93.5:3301",])

@everywhere include("/workspace/distributed_clustering/julia/src/dsnn_IO.jl")
@everywhere include("/workspace/distributed_clustering/julia/src/dsnn_KNN.jl")
@everywhere include("/workspace/distributed_clustering/julia/src/dsnn_SNN.jl")
@everywhere include("/workspace/distributed_clustering/julia/src/dsnn_Master.jl")

using PyCall
@pyimport clustering_scores as cs #clustering_scores.py must be in the path.
using JLD

In [ ]:
results = Dict{String,Any}();
DATA_PATH = "/workspace/cure_large.dat";
partitions = DSNN_Master.generate_partitions(length(workers()), 8100); # N must be extracted from the data.
DSNN_Master.start(results, DATA_PATH, partitions, 0.05, 5, 50, 0.5, 40);

In [ ]:
Stats.countmap(results["stage2_labels"])

In [ ]:
#using Plots
#pyplot()
#D = DSNN_IO.sparseMatFromFile(DATA_PATH, l2normalize=false);

colors = fill(1, size(D,2));
colors[results["stage1_corepoints"]] = 2;
scatter(D[1,:], D[2,:], marker=(0.05,:o,4), legend=false, fmt = :png)
scatter!(D[1,results["stage1_corepoints"]], D[2,results["stage1_corepoints"]], marker=(0.8,:o,4), legend=false, fmt = :png)